In [0]:
# Installer les bibliothèques nécessaires
# %pip install requests pydicom pillow

In [0]:
# !pip install opencv-python

In [0]:
import os
import requests
import zipfile
import shutil
import io
from pyspark.dbutils import DBUtils
import sys
import pydicom
import matplotlib.pyplot as plt
import config
import importlib
import numpy as np
from pyspark.sql import SparkSession
import tensorflow as tf
import cv2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout

In [0]:
# Ajoute le chemin où se trouve config.py
sys.path.append("/Workspace/Users/alban.daniel@amaris.com/")

# Vérifie si config.py existe avant de l'importer
config_path = "/Workspace/Users/alban.daniel@amaris.com/config.py"

if not os.path.exists(config_path):
    with open(config_path, "w") as f:
        f.write('BASE_URL = "https://services.cancerimagingarchive.net/services/v4/TCIA/query"\n')
        f.write('COLLECTION_NAME = "LIDC-IDRI"\n')

import config  # Maintenant, ça fonctionnera toujours !

# Vérification rapide
print(f"✅ Config chargé avec BASE_URL: {config.BASE_URL}")

In [0]:
# 📌 Initialisation de dbutils (nécessaire en dehors d'un notebook Databricks)
try:
    dbutils = DBUtils(spark)
except NameError:
    print("⚠ dbutils non disponible en dehors de Databricks.")

In [0]:
display(dbutils.fs.ls("file:/Workspace/Users/alban.daniel@amaris.com/"))

In [0]:
config_path = "/Workspace/Users/alban.daniel@amaris.com/config.py"

# 🔍 Lire le contenu existant
with open(config_path, "r") as f:
    config_content = f.read()

# ✍ Ajouter COLLECTION_NAME si elle est absente
if "COLLECTION_NAME" not in config_content:
    with open(config_path, "a") as f:  # "a" = append (ajout)
        f.write('\nCOLLECTION_NAME = "LIDC-IDRI"\n')

print("✅ COLLECTION_NAME ajouté à config.py !")

In [0]:
importlib.reload(config)  # ✅ Recharge le module pour être sûr

# Vérification
print(f"✅ BASE_URL: {config.BASE_URL}")
print(f"✅ COLLECTION_NAME: {config.COLLECTION_NAME}")  # Doit s'afficher maintenant !

In [0]:
# URL de l'API pour récupérer les patients
patients_url = f"{config.BASE_URL}/getPatient"

# Paramètres de la requête
params = {"Collection": config.COLLECTION_NAME}

# Envoi de la requête
response = requests.get(patients_url, params=params)

# Vérification et affichage des patients
if response.status_code == 200:
    patients = response.json()
    print(f"Nombre total de patients : {len(patients)}")
    print("Quelques patients :", display(patients[:5]))  # Afficher les 5 premiers
else:
    print("❌ Erreur lors de la récupération des patients :", response.text)

In [0]:
series_url = f"{config.BASE_URL}/getSeries"
params = {"PatientID": "LIDC-IDRI-0001"}  # Remplace par d'autres si besoin

response = requests.get(series_url, params=params)

if response.status_code == 200:
    series = response.json()
    print(f"Nombre de séries pour LIDC-IDRI-0001 : {len(series)}")
    print("Aperçu des séries :", display(series[:5]))  # Affiche les 5 premières
else:
    print("❌ Erreur lors de la récupération des séries :", response.text)

In [0]:
# URL pour récupérer les images d'une série
image_url = f"{config.BASE_URL}/getImage"
params = {"SeriesInstanceUID": "1.3.6.1.4.1.14519.5.2.1.6279.6001.179049373636438705059720603192"}  # Série CT

# Envoi de la requête
response = requests.get(image_url, params=params, stream=True)

# Vérification et téléchargement
if response.status_code == 200:
    with open("test_dicom.zip", "wb") as f:
        for chunk in response.iter_content(chunk_size=8192):
            f.write(chunk)
    print("✅ Téléchargement réussi : test_dicom.zip")
else:
    print("❌ Erreur lors du téléchargement :", response.text)

In [0]:
# 📌 Étape 1 : Décompression
with zipfile.ZipFile("test_dicom.zip", "r") as zip_ref:
    zip_ref.extractall("dicom_images")  # Décompresser dans un dossier

# 📌 Étape 2 : Lister les fichiers extraits
dicom_files = [f for f in os.listdir("dicom_images") if f.endswith(".dcm")]
print(f"📂 Nombre de fichiers DICOM extraits : {len(dicom_files)}")

# 📌 Étape 3 : Charger et afficher une image DICOM
if dicom_files:
    dicom_path = os.path.join("dicom_images", dicom_files[0])
    dicom_data = pydicom.dcmread(dicom_path)

    # Affichage de l'image
    plt.imshow(dicom_data.pixel_array, cmap="gray")
    plt.title(f"Image DICOM : {dicom_files[0]}")
    plt.axis("off")
    plt.show()
else:
    print("❌ Aucun fichier DICOM trouvé !")

In [0]:
print(dicom_data)  # Affiche toutes les métadonnées de l’image

In [0]:
# 📂 Répertoire où sont stockées les images DICOM
dicom_dir = "dicom_images"
dicom_files = [f for f in os.listdir(dicom_dir) if f.endswith(".dcm")]

# 🔍 Stocker les anomalies détectées
anomalies = []

for file in dicom_files:
    try:
        dicom_path = os.path.join(dicom_dir, file)
        dicom_data = pydicom.dcmread(dicom_path)
        pixel_array = dicom_data.pixel_array  # Convertir l’image en array

        # 📏 Vérifier la taille de l'image
        height, width = pixel_array.shape
        if height < 64 or width < 64:
            anomalies.append((file, "Image trop petite"))

        # 🎨 Vérifier les valeurs des pixels
        mean_pixel = np.mean(pixel_array)
        if mean_pixel < 5:  # Image très sombre
            anomalies.append((file, "Image quasi noire"))
        elif mean_pixel > 250:  # Image très claire
            anomalies.append((file, "Image quasi blanche"))

        # 🔍 Vérifier si des métadonnées clés sont manquantes
        required_fields = ["Modality", "BodyPartExamined"]
        for field in required_fields:
            if not hasattr(dicom_data, field) or getattr(dicom_data, field) == "":
                anomalies.append((file, f"Métadonnée manquante: {field}"))

    except Exception as e:
        anomalies.append((file, f"Erreur de lecture DICOM: {str(e)}"))

# 📢 Affichage des anomalies
if anomalies:
    print("⚠️ Anomalies détectées :")
    for anomaly in anomalies[:10]:  # Afficher les 10 premières
        print(f"📌 {anomaly[0]} ➝ {anomaly[1]}")
else:
    print("✅ Aucune anomalie détectée !")

# 📸 Affichage d'images suspectes
if anomalies:
    fig, axs = plt.subplots(1, min(5, len(anomalies)), figsize=(15, 5))
    for i, (file, issue) in enumerate(anomalies[:5]):
        dicom_path = os.path.join(dicom_dir, file)
        dicom_data = pydicom.dcmread(dicom_path)
        axs[i].imshow(dicom_data.pixel_array, cmap="gray")
        axs[i].set_title(issue)
        axs[i].axis("off")
    plt.show()

In [0]:
dbutils.fs.mkdirs("dbfs:/mnt/dicom_data")  # Crée le dossier si nécessaire

local_dicom_path = "dicom_images"
dbfs_dicom_path = "dbfs:/mnt/dicom_data"

copied_files = 0

for file in os.listdir(local_dicom_path):
    if file.endswith(".dcm"):
        local_file_path = f"file:{os.path.abspath(os.path.join(local_dicom_path, file))}"
        dbfs_file_path = f"{dbfs_dicom_path}/{file}"

        try:
            dbutils.fs.cp(local_file_path, dbfs_file_path)
            copied_files += 1
        except Exception as e:
            print(f"❌ Erreur lors de la copie de {file} : {e}")

print(f"✅ {copied_files} fichiers copiés vers {dbfs_dicom_path}")

# Vérifier si les fichiers sont bien copiés
display(dbutils.fs.ls(dbfs_dicom_path))

In [0]:
# %pip install tensorflow

In [0]:
# Taille des images optimisée pour CPU
IMG_SIZE = 64  
EPOCHS = 3  

# Chemin vers DBFS
DBFS_PATH = "dbfs:/mnt/dicom_data/"

def load_dicom_images(dbfs_path):
    images = []
    labels = []  # À adapter selon tes classes

    # Utilisation de dbutils.fs.ls() pour lister les fichiers
    file_list = [f.path for f in dbutils.fs.ls(dbfs_path) if f.path.endswith(".dcm")]

    for file_path in file_list:
        try:
            # Convertir le chemin DBFS en chemin accessible pour Python
            local_path = "/dbfs" + file_path[5:]  # Remplace "dbfs:/" par "/dbfs"
            
            dicom_data = pydicom.dcmread(local_path)
            pixel_array = dicom_data.pixel_array

            # Normalisation et redimensionnement
            img_resized = cv2.resize(pixel_array, (IMG_SIZE, IMG_SIZE))
            img_normalized = img_resized / 255.0

            images.append(img_normalized)
            labels.append(0)  # ⚠️ Remplace "0" par la vraie classe de l'image

        except Exception as e:
            print(f"❌ Erreur sur {file_path}: {e}")

    images = np.array(images).reshape(-1, IMG_SIZE, IMG_SIZE, 1)  # Format CNN
    labels = np.array(labels)
    
    return images, labels

# Chargement des données
X, y = load_dicom_images(DBFS_PATH)

# Conversion des labels en one-hot encoding
NUM_CLASSES = len(np.unique(y))
y = to_categorical(y, NUM_CLASSES)

# Séparation train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"✅ Chargement terminé : {len(X_train)} images pour l'entraînement, {len(X_test)} pour le test.")